In [30]:
import tkinter as tk
from tkinter import ttk, filedialog
import pandas as pd

def calcular_media_situacao(df):
    df['Média'] = df[['Nota 1','Nota 2','Nota 3','Nota 4']].mean(axis=1)
    
    def definir_situacao(row):
        if row['Faltas'] > 10:
            return 'Reprovado por Faltas'
        elif row['Média'] >= 7:
            return 'Aprovado'
        elif row['Média'] < 2:
            return 'Reprovado por Notas'
        else:
            return 'Recuperação'
        
    df['Situação'] = df.apply(definir_situacao,axis=1)
    
    return df

def carregar_dados():
    global df_original

    df_original = pd.read_excel('notas_estudantes.xlsx', sheet_name='Dados')

    df_original = calcular_media_situacao(df_original)

    turmas = df_original["Turma"].unique()

    for turma in turmas:
        aba = ttk.Frame(caderno)
        caderno.add(aba,text=turma)
        abas[turma] = aba
    
    aplicar_filtro()

def colorir_situacao(tree,col_index):
    for item in tree.get_children():
        situacao = tree.item(item,"values")[col_index]
        
        if situacao == 'Aprovado':
            tree.item(item, tags=('aprovado',))
        elif situacao == 'Recuperação':
            tree.item(item, tags=('recuperação',))
        elif situacao == 'Reprovado por Notas':
            tree.item(item, tags=('reprovado_notas',))
        elif situacao == 'Reprovado por Faltas':
            tree.item(item, tags=('reprovado_faltas',))

    tree.tag_configure("aprovado", background="#d4edda",foreground="#155724")
    tree.tag_configure("recuperação", background="#fff3cd",foreground="#856404")
    tree.tag_configure("reprovado_notas", background="#f8d7da",foreground="#721c24")
    tree.tag_configure("reprovado_faltas", background="#f5c6cb",foreground="#721c24")    

def aplicar_filtro():
    termo = entrada_filtro.get().lower()

    for turma, aba in abas.items():
        for widget in aba.winfo_children():
            widget.destroy()
        
        df_filtrados = df_original[df_original.apply(lambda row: termo in row.to_string(index=False).lower(),axis=1)]

        df_turma = df_filtrados[df_filtrados['Turma']==turma]

        tree = ttk.Treeview(aba,
                            columns=list(df_turma.columns),
                            show='headings')
        tree.pack(expand=True,fill="both")
        
        larguras_colunas = {
            'Nome': 150,
            'Turma': 80,
            'Nota 1': 60,
            'Nota 2': 60,
            'Nota 3': 60,
            'Nota 4': 60,
            'Faltas': 60,
            'Média': 80,
            'Situação': 120,
        }

        for col in df_turma.columns:
            tree.heading(col,text=col)
            tree.column(col,
                        anchor='center',
                        width=larguras_colunas.get(col, 100))
            
        for indice, linha in df_turma.iterrows():
            tree.insert("",
                        "end",
                        values=list(linha))
            
        colorir_situacao(tree,list(df_turma.columns).index("Situação"))

        dados_filtrados[turma] = df_turma

def exportar_dados_filtrados_aba_ativa():
    aba_ativa = caderno.tab(caderno.select(),"text")
    df_exportar = dados_filtrados.get(aba_ativa)
    if df_exportar is not None:
        arquivo = filedialog.asksaveasfilename(defaultextension=".xlsx",
                                               filetypes=[("Excel files", "*.xlsx")])
    
        if arquivo:
            df_exportar.to_excel(arquivo, index=False)


def exportar_todas_aba():
    arquivo = filedialog.asksaveasfilename(defaultextension=".xlsx",
                                               filetypes=[("Excel files", "*.xlsx")])
    if arquivo:
        with pd.ExcelWriter(arquivo) as writer:
            for turma in abas.keys():
                df_exportar = dados_filtrados.get(turma,
                                                  pd.DataFrame())
                df_exportar.to_excel(writer,
                                     sheet_name=turma,
                                     index=False)

janela = tk.Tk()
janela.title("Gestão de Notas dos Estudantes")
janela.geometry("1024x400")

titulo = tk.Label(janela, text="Projeto: Gestão de Notas dos Estudantes",
                  font=("Arial", 18, "bold"),
                  bg="#343a40",
                  fg="white")
titulo.pack(fill="x")

frame_filtro = tk.Frame(janela,bg="#f8f9fa")
frame_filtro.pack(pady=10,fill='x')

label_filtro = tk.Label(frame_filtro,
                        text="Filtrar: ",
                        bg="#f8f9fa",
                        font=("Arial",12))
label_filtro.pack(side="left",padx=10)

entrada_filtro = tk.Entry(frame_filtro,font=("Arial",12))
entrada_filtro.pack(side="left",padx=10, fill='x', expand=True)
entrada_filtro.bind("<KeyRelease>",lambda event: aplicar_filtro())

caderno = ttk.Notebook(janela)
caderno.pack(expand=True, fill='both')

abas = {}
dados_filtrados = {}

frame_botoes = tk.Frame(janela,
                        bg="#f8f9fa")
frame_botoes.pack(pady=10,fill='x')

botao_exportar_aba = tk.Button(frame_botoes,
                               text="Exportar Dados Filtrados da Aba Ativa",
                               command=exportar_dados_filtrados_aba_ativa,
                               bg="#007bff",
                               fg="white",
                               font=("Arial", 12, "bold"))
botao_exportar_aba.pack(side="left",padx=10,pady=5)

botao_exportar_todas = tk.Button(frame_botoes,
                               text="Exportar Todas as Abas",
                               command=exportar_todas_aba,
                               bg="#28a745",
                               fg="white",
                               font=("Arial", 12, "bold"))
botao_exportar_todas.pack(side="left",padx=10,pady=5)

carregar_dados()

janela.mainloop()